# 311 Data Cleaning

This notebook will analyze the the [csv file from empowerla.org](https://data.lacity.org/City-Infrastructure-Service-Requests/MyLA311-Service-Request-Data-2021/97z7-y5bt) pulled on Dec 02 2021.  The analysis, and any changes, should align with [the data transforms](https://github.com/hackforla/311-data/blob/dev/docs/data_loading.md) for the H4LA 311-data project.

**Outputs** of the analysis:
  * "Clean" (my definition of clean!) csv file
  * A geodataframe, using the cleaned csv, saved as a shape file and geojson

**Steps** in the process are:

1.  Read and analyze structure and content
2.  Explore data values to determine which fields have valuable information
3.  Rename columns on the initial dataframe
4.  dtype conversions 
5.  Create new dataframe with selected columns
6.  Create a geodataframe for spatial processing
7.  Save both the clean subset of the csv file as csv and the geodataframe as shape file

# 1 - Read and analyze

This is a generic look at the csv file.  Pretty standard fare.  Basics include:

  * shape
  * info
  * iloc for one, example instance
  * hack to summarize the null value counts
  
I doubt there is much need for explanation on this step.  This is the first step in deciding which columns provide **good features** for later analysis.

**Note** - I'm modifying things a bit for first release.  I'm using pd.read_csv with the actual url for 311 download.  Makes for one less large file.  You can download and use the second pd.read_csv if you prefer.

In [ ]:
%run start.py

In [ ]:
%%time
myla311_df = pd.read_csv('https://data.lacity.org/api/views/97z7-y5bt/rows.csv', low_memory=False)

Uncomment and use this if you downloaded the file.

In [ ]:
#myla311_df = pd.read_csv('../data/311/MyLA311_Service_Request_Data_2021.csv', 
#                         low_memory=False)
#                        #parse_dates=['CreatedDate', 'UpdatedDate', 'ServiceDate', 'ClosedDate'])

Basic descriptions of the dataframe (df) look at the **shape** and **info** for counts and dtypes.

In [ ]:
myla311_df.shape

In [ ]:
myla311_df.RequestType.value_counts()

In [ ]:
myla311_df.info()

Not sure why, but I always look at the **27**th value first?

Here are the values for that row.

In [ ]:
myla311_df.iloc[27]

I use this simple hack to see percentage of null counts in each column of the df.  High numbers, i.e. **MobileOS** at **73%** probably don't provide much value (IMHO).  

It's also important to see the ones that are 0.00, i.e. they all have a value.

In [ ]:
(myla311_df.isnull().mean() * 100).round(2)

At this stage we have some basic views of the data.  We can combine these to determine which columns (variables) could bring value to further analysis.

Next we'll look at how we might combine this info.  Remember, our goal is to identify the variables that will provide value for upstream analysis.

# 2 - Explore dataframe values

OK.  This section is a bit of a **digression!**  I'm using some basic jupy widgetry to combine the outputs from the first section.

30K foot view of this section:

   1.  Create Output widgets for info and null value displays
   2.  Build interactive function to display value_counts for selected column name
   3.  Combine via HBox widget to show them together

In [ ]:
info_df = Output(layout={'border': '1px solid black',
                            'width': '50%'})

null_info = Output(layout={'border': '1px solid black',
                            'width': '30%'})

with info_df:
    display(HTML('<center><b>myla311_df info()</b></center>'))
    display(myla311_df.info())
    
with null_info:
    display(HTML('<center><b>normalized info()</b></center>'))
    display((myla311_df.isnull().mean() * 100).round(2))

from ipywidgets import interact, interactive

def examine_value_counts(col):
    display(myla311_df[col].value_counts())
    
    
w = interactive(examine_value_counts, col=list(myla311_df.columns))
w;

At this point use the info and null value displays to identify columns to explore.  Couple of examples to explore:

  1.  Note that RequestType is 0.00 null values (i.e. all rows have a value).  Check out the value_counts by selecting RequestType.
  2.  Check out CD/CDMember and NC/NCName.  Need to do something about the floats?
  3.  ...  You can choose some to explore!

In [ ]:
HBox([info_df, null_info, w])

# 3 - Rename columns

mapping_311 is the dictionary I created in my editor.  I edited the dictionary generated from this comprehension:

```python
{v[0], v[1] for v in zip(myla311_df.columns.to_list(), myla311_df.columns.to_list()}
```       

I tried to make the names more pythonic.

Here's my mapping dict.

In [ ]:
mapping_311 = {'SRNumber': 'SRNumber',
               'CreatedDate': 'created_dt',
               'UpdatedDate': 'updated_dt',
               'ActionTaken': 'ActionTaken',
               'Owner': 'owner',
               'RequestType': 'request_type',
               'Status': 'Status',
               'RequestSource': 'RequestSource',
               'MobileOS': 'MobileOS',
               'Anonymous': 'Anonymous',
               'AssignTo': 'AssignTo',
               'ServiceDate': 'service_dt',
               'ClosedDate': 'closed_dt',
               'AddressVerified': 'AddressVerified',
               'ApproximateAddress': 'ApproximateAddress',
               'Address': 'address',
               'HouseNumber': 'HouseNumber',
               'Direction': 'Direction',
               'StreetName': 'street',
               'Suffix': 'Suffix',
               'ZipCode': 'zip_code',
               'Latitude': 'latitude',
               'Longitude': 'longitude',
               'Location': 'location',
               'TBMPage': 'TBMPage',
               'TBMColumn': 'TBMColumn',
               'TBMRow': 'TBMRow',
               'APC': 'APC',
               'CD': 'cd',
               'CDMember': 'cd_member',
               'NC': 'nc',
               'NCName': 'nc_name',
               'PolicePrecinct': 'precinct'
}

I'm going to hold off on renaming for a bit.  

**Note to self** - So in the next section I'm adding new columns ... how does that relate to this dictionary?  Maybe I should do this process after next section?

# 4 - dtype conversions and new columns

Conversions (for starters):

> * dates - date information is object dtype (str) so convert to python dtime
> * ids - floats, convert to int

New colums are added for duration of the request.

> * days_to_service
> * days_to_close
> * days_to_update

For the duration code I'm borrowing [work from the 311 data science team](https://colab.research.google.com/drive/1LvuuPDWPGC6g3WOLHOk806ocQbYAJ5qj#scrollTo=g-H5Fq91BSYW).  Chelsey shared this collab link (thanks Chelsey).  
Honestly, I'm not sure what to do with these yet?

In [ ]:
myla311_df['CreatedDate'] = pd.to_datetime(myla311_df['CreatedDate'])
myla311_df['ServiceDate'] = pd.to_datetime(myla311_df['ServiceDate'])
myla311_df['ClosedDate'] = pd.to_datetime(myla311_df['ClosedDate'])
myla311_df['UpdatedDate'] = pd.to_datetime(myla311_df['UpdatedDate'])

In [ ]:
myla311_df['days_to_service'] = (myla311_df.ServiceDate - myla311_df.CreatedDate).astype('timedelta64[D]')
myla311_df['days_to_close'] = (myla311_df.ClosedDate - myla311_df.CreatedDate).astype('timedelta64[D]')
myla311_df['days_to_update'] = (myla311_df.UpdatedDate - myla311_df.CreatedDate).astype('timedelta64[D]')

In [ ]:
myla311_df.info()

So geopandas driver for esri shape doesn't support dt

In [ ]:
#myla311_df['CreatedDate'] = myla311_df['CreatedDate'].astype(str)

#myla311_df['ServiceDate'] = myla311_df['ServiceDate'].astype(str)
#myla311_df['ClosedDate'] = myla311_df['ClosedDate'].astype(str)
#myla311_df['UpdatedDate'] = myla311_df['UpdatedDate'].astype(str)

Since there are NaN's in NC and CD I'm using this idea https://stackoverflow.com/questions/21287624/convert-pandas-column-containing-nans-to-dtype-int

In [ ]:
myla311_df['NC'] = myla311_df['NC'].astype('Int64')
myla311_df['CD'] = myla311_df['CD'].astype('Int64')

In [ ]:
myla311_df.info()

# 5 - Create New Dataframe

I'm cheating a bit.  I have some idea of the types of analysis I want to do later, so that is driving which columns I want to include for now.  Not sure why I'm keeping location?

Steps in this section:

1.  create new df from myla311_df
2.  use the mapping dictionary to rename the columns
3.  Use the list of columns to select for a new df.

The result of this step is the new311_df.  myla311 is still intact, albeit with cleaned columns.

In [ ]:
column_subset = ['SRNumber',
                 'created_dt',
                 'updated_dt',
                 'owner',
                 'request_type',
                 'service_dt',
                 'closed_dt',
                 'address',
                 'street',
                 'zip_code',
                 'latitude',
                 'longitude',
                 'location',
                 'APC',
                 'cd',
                 'cd_member',
                 'nc',
                 'nc_name',
                 'precinct',
                 'days_to_service',
                 'days_to_close',
                 'days_to_update'
]

In [ ]:
new311_df = myla311_df.rename(columns=mapping_311)

In [ ]:
new311_df = new311_df[column_subset]

In [ ]:
new311_df.info()

So this df looks ok?

**Note to self** - really need to abstract the widgets above so we can apply the same viz to this df!

Save the clean csv file.

In [ ]:
%%time
new311_df.to_csv('../data/311/clean311.csv', index=False)

# 6 - Create the GeoDataframe

Use the cleaned csv file to generate a geodataframe then save (shape, geojson, zip).

The only _tricks_ here are:

1. Make sure the row has valid geo content
2. Create the geometry as wkt Point feature (note Point get's us to x, y so need to use longitude, latitiude order)
3. Save the geodataframe in multiple formats
4. Because geopandas uses ESRI driver and it's limited to 10 character names have to jump through a hoop

At the end of this section there are four addtional files written to ../data/311.

In [ ]:
(new311_df.location.isnull().sum()) / len(new311_df)

An extremely small number of rows don't have location so just toss 'em.

In [ ]:
geocodeable311_df = new311_df[new311_df.location.notnull()]

In [ ]:
new311_gdf = GeoDataFrame(geocodeable311_df,
                          geometry = [Point(xy[0], xy[1]) for xy in zip(geocodeable311_df['longitude'], geocodeable311_df['latitude'])])

In [ ]:
new311_gdf.info()

In [ ]:
new311_gdf['created_dt'] = new311_gdf['created_dt'].astype(str)

new311_gdf['service_dt'] = new311_gdf['service_dt'].astype(str)
new311_gdf['closed_dt'] = new311_gdf['closed_dt'].astype(str)
new311_gdf['updated_dt'] = new311_gdf['updated_dt'].astype(str)

In [ ]:
%%time
new311_gdf.to_file('../data/311/clean311-geo.shp', index=False)
#new311_gdf.to_file('/home/mcmorgan/for-sarah/clean311-geo.shp', index=False)

In [ ]:
%%time
new311_gdf.to_file('../data/311/clean311.geojson', driver='GeoJSON')
#new311_gdf.to_file('/home/mcmorgan/for-sarah/clean311.geojson', driver='GeoJSON')

In [ ]:
%%time
new311_gdf.to_file('../data/311/clean311-geo.zip', index=False)

At this point four new data sets have been added to ../data/311

1.  clean311.csv - the subset with columns used to generate the geodataframes
2.  clean311-geo.shp - There are three other files for the shapefile spec, but this is the one to read back in
3.  clean311.geojson - The geojson version of the data frame
4.  clean311-geo.zip - A zipped shape folder

I've added a function in src/utils.py, read_new311_shape to read the shape/zip files and do the dtype conversions.

In [ ]:
!ls -alh --time-style=+%D ../data/311 | grep $(date +%D)

went out to the command line and zipped these files up.  Hoping to fit them into the repo with lfs.

In [ ]:
!ls -alh --time-style=+%D ../data/311 | grep $(date +%D)